In [18]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
from selenium.webdriver.common.proxy import Proxy, ProxyType

CHROMEDRIVER_PATH = './drivers/chromedriver.exe' # Windows는 chromedriver.exe로 변경
WINDOW_SIZE = "1920,1080"
 
chrome_options = Options()
chrome_options.add_argument( "--headless" )     # 크롬창이 열리지 않음
chrome_options.add_argument( "--no-sandbox" )   # GUI를 사용할 수 없는 환경에서 설정, linux, docker 등
chrome_options.add_argument( "--disable-gpu" )  # GUI를 사용할 수 없는 환경에서 설정, linux, docker 등
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")

chrome_options.add_argument(f"--window-size={ WINDOW_SIZE }")
chrome_options.add_argument('Content-Type=application/json; charset=utf-8')

In [19]:
from tqdm import tqdm

In [20]:
links = pd.read_csv('./data/음식점.csv')

In [21]:
# PROXY = "202.169.255.7:8181" # IP:Port

# webdriver.DesiredCapabilities.CHROME['proxy']={
#     "httpProxy":PROXY,
#     "ftpProxy":PROXY,
#     "sslProxy":PROXY,
#     "noProxy":None,
#     "proxyType":"MANUAL",
#     "autodetect":False
# }
driver = webdriver.Chrome( executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options )
driver.get("https://www.google.com")

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [22]:
def get_reviews():
    reviews = []
    try:
        page_cnt = 2
        btn = None
        while True:
            sleep(0.5)
            for i in driver.find_element_by_class_name('list_evaluation').find_elements_by_css_selector('li'):
                review = {
                    "profile": i.find_element_by_class_name('profile_info').get_attribute('data-ori'),
                    "rate": i.find_element_by_class_name('num_rate').text,
                    "content": i.find_element_by_css_selector('div.comment_info > p > span').text,
                    "user_name": i.find_element_by_css_selector('div.comment_info > div > a.link_user').get_attribute('data-username'),
                    "write_time": i.find_element_by_css_selector('div.comment_info > div > span.time_write').text
                }
                reviews.append(review)

            # next 버튼
            if page_cnt==6:
                next_btn = driver.find_element_by_class_name('btn_next')
                if next_btn:
                    raise
                next_btn.click()
                page_cnt = 2
                sleep(0.5)

            # btn 체크
            for i in driver.find_elements_by_css_selector("#mArticle > div.cont_evaluation > div.evaluation_review > div > a"):
                if int(i.text)==page_cnt:
                    page_cnt += 1
                    btn = i
                    break
            else:
                raise
            btn.click()
    except:
        pass

    return reviews

In [23]:
def check_exists_by_xpath(path):
    try:
        return driver.find_element_by_xpath(path).text
    except:
        return None

In [24]:
def check_exists_by_class_name(name):
    try:
        return driver.find_element_by_class_name('txt_location').text
    except:
        return None

In [27]:
result = []
except_list = []
cnt = 0
for link in tqdm(temp_links):
    driver.get(link)
    sleep(3)
    reviews = get_reviews()
    try:
        data = {
            "id": int(link.replace('https://place.map.kakao.com/','')),
            "title": check_exists_by_xpath('//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2'),
            "address": check_exists_by_xpath('//*[@id="mArticle"]/div[1]/div[2]/div[1]/div/span[1]'),
            "phone": check_exists_by_xpath('//*[@id="mArticle"]/div[1]/div[2]/div[2]/div/div/span/span[1]'),
            "category": '음식점 | ' + check_exists_by_class_name('txt_location'),
            "reviews": reviews,
        }
        result.append(data)
    except:
        cnt += 1
        print(link)
        except_list.append(link)
        if cnt>=10:
            break

 19%|█▉        | 3514/18340 [3:38:17<15:04:11,  3.66s/it]

https://place.map.kakao.com/9983951


 19%|█▉        | 3515/18340 [3:38:20<14:57:34,  3.63s/it]

https://place.map.kakao.com/11288696


 19%|█▉        | 3516/18340 [3:38:24<14:52:14,  3.61s/it]

...

https://place.map.kakao.com/1601004803


 19%|█▉        | 3522/18340 [3:38:45<14:39:50,  3.56s/it]

https://place.map.kakao.com/231141398


 19%|█▉        | 3522/18340 [3:38:49<15:20:37,  3.73s/it]

https://place.map.kakao.com/21363057


In [23]:
# 0 ~ 3582 까지 했음....

In [8]:
# 여기부서부터!!!
temp = links['0'].loc[3583:]

In [43]:
temp1 = links['0'].loc[3583+759+895+62+2:]

In [30]:
temp2 = links['0'].loc[10719:]
temp2

10719       https://place.map.kakao.com/9983951
10720      https://place.map.kakao.com/11288696
10721      https://place.map.kakao.com/11566046
10722      https://place.map.kakao.com/21159390
10723    https://place.map.kakao.com/1321108872
                          ...                  
25541      https://place.map.kakao.com/16456661
25542    https://place.map.kakao.com/1147685619
25543       https://place.map.kakao.com/8012297
25544     https://place.map.kakao.com/509640969
25545      https://place.map.kakao.com/16663645
Name: 0, Length: 14827, dtype: object

In [28]:
links[links['0']=="https://place.map.kakao.com/9983951"]

,Unnamed: 0,0
10719,10719,https://place.map.kakao.com/9983951


In [32]:
len(result)

3513

In [55]:
df = pd.DataFrame(data=result, columns=["id", "title", "address", "phone", "reviews"])
df.to_csv('data_PC방.csv',sep="|")

In [52]:
import json

In [ ]:
toJson(mnet_dict)

In [33]:
with open('음식점3513.json', 'w', encoding='utf-8') as file :
        json.dump(result, file, ensure_ascii=False, indent='\t')